In [1]:
import numpy 
import os
import glob
import pandas as pd
import librosa
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import seaborn as sns
from keras.preprocessing import image
from tensorflow.keras.models import load_model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/Audio_Classification-MLSP'
data_path=os.path.join(path, "train")
label_path=path
data=pd.read_csv(os.path.join(label_path, "annotations.csv"))

In [4]:
inputs=[]
files=data['fname']
for file in files:
  arr=numpy.load(os.path.join(data_path, file))
  m,n,o=arr.shape
  print(m, n, o)
  arr.resize(n,o)
  # Convert a power spectrogram (amplitude squared) to decibel (dB) units
  mel_spect = librosa.power_to_db(arr, ref=numpy.max)
  inputs.append(mel_spect)

1 128 314
1 128 2504
1 128 236
1 128 1070
1 128 272
1 128 70
1 128 664
1 128 1421
1 128 432
1 128 200
1 128 1211
1 128 2512
1 128 52
1 128 868
1 128 106
1 128 34
1 128 2512
1 128 2584
1 128 72
1 128 32
1 128 44
1 128 194
1 128 100
1 128 224
1 128 1506
1 128 269
1 128 518
1 128 586
1 128 100
1 128 194
1 128 41
1 128 359
1 128 282
1 128 94
1 128 314
1 128 1701
1 128 240
1 128 55
1 128 2512
1 128 1114
1 128 230
1 128 47
1 128 272
1 128 309
1 128 259
1 128 1512
1 128 2357
1 128 645
1 128 52
1 128 57
1 128 340
1 128 369
1 128 475
1 128 209
1 128 1534
1 128 244
1 128 1701
1 128 340
1 128 908
1 128 307
1 128 42
1 128 50
1 128 52
1 128 268
1 128 47
1 128 216
1 128 197
1 128 716
1 128 64
1 128 494
1 128 34
1 128 162
1 128 29
1 128 436
1 128 445
1 128 170
1 128 40
1 128 307
1 128 434
1 128 1628
1 128 280
1 128 48
1 128 50
1 128 56
1 128 257
1 128 381
1 128 254
1 128 1628
1 128 100
1 128 451
1 128 760
1 128 1471
1 128 100
1 128 1904
1 128 325
1 128 475
1 128 381
1 128 51
1 128 204
1 128 50
1 128 

In [5]:
maxs=0
for input in inputs:
  m,n=input.shape
  if maxs<n:
      maxs=n
# add padding in the give .npy files
features=[]
for a in inputs:
  mat=numpy.pad(a, [(0, 0), (0, maxs-a[0].size)], mode='constant', constant_values=0)
  mat=mat.reshape((mat.shape[0], mat.shape[1], 1))
  features.append(mat)
maxs=0
features=numpy.array(features)
for a in features:
  print(a.shape)

(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584, 1)
(128, 2584

In [6]:
data['label'].describe()

count     1000
unique      10
top       Bark
freq       100
Name: label, dtype: object

In [7]:
i=0
for mat in features:
    features[i]= tf.convert_to_tensor(features[i])
    i=i+1

In [8]:
X=features
y=data["label"]
y.shape

(1000,)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40, shuffle="true")

print(type(y_test))
val_label=pd.Series(list(y_test))
val_label.value_counts()

<class 'pandas.core.series.Series'>


Bark                                     22
Walk_and_footsteps                       22
Doorbell                                 22
Crying_and_sobbing                       21
Siren                                    20
Knock                                    20
Vehicle_horn_and_car_horn_and_honking    20
Microwave_oven                           19
Shatter                                  17
Meow                                     17
dtype: int64

In [10]:
# split data equally for every class 

In [11]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
test_encoder = LabelEncoder()
test_ = test_encoder.fit_transform(y_test)
to_onehot=to_categorical(test_)
y_test=to_onehot
train_encoder = LabelEncoder()
train_ = train_encoder.fit_transform(y_train)
to_onehot=to_categorical(train_)
y_train=to_onehot

In [12]:
pool_size = (2, 2)
kernel_size = (3, 3)
input_shape = (128, 2584, 1)
num_classes = 10
# tensorflow.keras.backend.clear_session()

model = tf.keras.models.Sequential([
    #first_convolution
    tf.keras.layers.Conv2D(32, kernel_size,
                padding="same", input_shape=input_shape),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # tf.keras.layers.Dropout(0.25),
    #second_convolution
    tf.keras.layers.Conv2D(64, kernel_size,
                                  padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    #third_convolution
    tf.keras.layers.Conv2D(128, kernel_size,
                                  padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    #fourth_convolution
    tf.keras.layers.Conv2D(256, kernel_size,
                                  padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    # tf.keras.layers.Dropout(0.25),
    #Fully connected 1st layer
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(256, activation='relu', kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(10, activation='softmax') 
]) 
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
# model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))

In [13]:
len(X_train)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 2584, 32)     320       
                                                                 
 batch_normalization (BatchN  (None, 128, 2584, 32)    128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 128, 2584, 32)     0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 1292, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 1292, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 64, 1292, 64)     2

In [14]:
from keras import backend as K
from sklearn.metrics import precision_score , recall_score
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [15]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
          loss=tf.keras.losses.CategoricalCrossentropy(),
          metrics=["accuracy", f1_m, precision_m, recall_m])

In [16]:
from keras.callbacks import ModelCheckpoint
filepath = '/content/drive/MyDrive/Audio_Classification-MLSP/my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
history=model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=160, verbose = 1, callbacks=[checkpoint])

Epoch 1/160
25/25 [==============================] - ETA: 0s - loss: 4.6750 - accuracy: 0.1100 - f1_m: 0.0588 - precision_m: 0.1135 - recall_m: 0.0413
Epoch 1: val_loss improved from inf to 2.58668, saving model to /content/drive/MyDrive/Audio_Classification-MLSP/my_best_model.epoch01-loss2.59.hdf5
25/25 [==============================] - 27s 571ms/step - loss: 4.6750 - accuracy: 0.1100 - f1_m: 0.0588 - precision_m: 0.1135 - recall_m: 0.0413 - val_loss: 2.5867 - val_accuracy: 0.1200 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/160
25/25 [==============================] - ETA: 0s - loss: 2.9942 - accuracy: 0.1412 - f1_m: 0.0711 - precision_m: 0.2013 - recall_m: 0.0437
Epoch 2: val_loss improved from 2.58668 to 2.52092, saving model to /content/drive/MyDrive/Audio_Classification-MLSP/my_best_model.epoch02-loss2.52.hdf5
25/25 [==============================] - 13s 528ms/step - loss: 2.9942 - accuracy: 0.1412 - f1_m: 0.0711 - precision_m: 0.2013 -

In [ ]:
from matplotlib import pyplot as plt
model.metrics_names
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
import numpy as np
accuracy = model.evaluate(X_test, y_test)
print('n', 'Test_Accuracy:-', accuracy[1])
pred=model.predict(X_test)
y_pred = np.argmax(pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print('confusion matrix')
print(confusion_matrix(y_true, y_pred))
#confusion matrix
f, ax = plt.subplots(figsize=(8,5))
sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt=".0f", ax=ax)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()

In [ ]:
filepath = '/content/drive/MyDrive/Audio_Classification-MLSP/my_best_model.epoch144-loss0.43.hdf5'
model = tf.keras.models.load_model(
    filepath, custom_objects={'f1_m':f1_m, 'precision_m':precision_m, 'recall_m':recall_m}, compile=True, options=None
)

In [ ]:
model.predict(X)